# Finding STk vs. Tk sequence determinants

In this example, we'll tackle a problem of multiple sequence alignemnt (MSA)
sequences' classification.

Protein kinases, or phosphate transferases, are regulatory enzymes ubiquitous
across all major known taxae. They catalyze transferring of phosphate moieties
from an ATP molecule to the carboxylic group of an amino acid residue. This
changes physico-chemical properties of a target protein altering it's overall
behaviour and functionality. Thus, they act as molecular controllers, deeply
embedded into regulating mechanisms governing complex cellular machinery.

Over the course of evolution, along with substrate specificity, PKs developed
a preference towards certain amino acid residues (Fig. 1). The principal group
of PKs that likely appeared initially were PKs transferring phosphate to serine
and threonine residues (STk). Later, another group emerged, specializing towards
the tyrosine residue (Tk).

| ![Phosphorylation](../fig/Phosphorylation.pdf "Phosphorylation reaction") |
|:--:|
| <b> Fig. 1. Phosphorylation of Ser/Thr and Tyr. Notice the dissimilarities between these two groups </b>|

What could be the sequence differences enabling such a specialization?
Could we find them using features selection machinery?

In 1994, Juswinder performed a conservation-based analysis of STk and Tk
sequences.
He discovered several MSA positions that, as structural evidence suggests, expliot
physico-chemical differences (e.g., volume) between Ser/Thr and Tyr.
For this, he used a scoring function that would spot positions with substantial
but different conservation between STk and Tk sequences in the same multiple
sequence alignment.

| ![PK domains alignment](../fig/juswinderA.png "Juswinder et al. MSA (A)") |
|:--:|
| <b> Fig. 2. Positions of STk ad Tk sub-alignment (note 201 and 203) </b>|

| ![PK domains alignment](../fig/juswinderB.png "Juswinder et al. MSA (B)") |
|:--:|
| <b> Fig. 2. Positions of STk and TK sub-alignment part2 (note 168 and 170) </b>|

Here, we'll attempt to recover positions highlighted by Juswinder on a larger
MSA built from reviewed [UniProt](https://www.uniprot.org) sequences.

## 1. Install dependencies

In [ ]:
# ! pip install lXtractor eBoruta ipywidgets xgboost optuna

## 2. Prepare data

In [ ]:
import gzip
import re
from collections import Counter
from io import StringIO, BytesIO
from itertools import chain

import pandas as pd
from lXtractor.core.chain import ChainSequence, ChainList
from lXtractor.ext.hmm import PyHMMer
from lXtractor.util.io import fetch_text
from lXtractor.variables.calculator import GenericCalculator
from lXtractor.variables.manager import Manager
from lXtractor.variables.sequential import PFP
from more_itertools import consume

### 2.1 Download and parse initial sequences and profile

The UniProt link is generated from the search API.
We are taking all reviewed proteins (Swiss-Prot database) belonging to the PK superfamily.

Link to Pfam points to the _PF00069_ -- a protein kinase domain HMM profile.

In [ ]:
LINK_UNIPROT = "https://rest.uniprot.org/uniprotkb/stream?fields=accession%2Cid%2Csequence%2Cft_domain%2Cprotein_families&format=tsv&query=%28%28family%3A%22protein%20kinase%20superfamily%22%29%29%20AND%20%28reviewed%3Atrue%29"
LINK_PFAM = 'https://www.ebi.ac.uk/interpro/wwwapi//entry/pfam/PF00069?annotation=hmm'

In [ ]:
df_up = pd.read_csv(StringIO(fetch_text(LINK_UNIPROT)), sep='\t')

In [ ]:
df_up.head()

Initialize profile as `PyHMMer` object internally interfacing homonimous library.
`PyHMMer` is an annotator class, annotating (subsetting) `ChainSequence`s using
HMM-derived domain boundaries.

In [ ]:
prof = PyHMMer(BytesIO(gzip.decompress(
    fetch_text(LINK_PFAM, decode=False)
)))

### 2.2 Convert sequences

Initialize `ChainSequence`s and wrap them into the `ChainList`.
The latter simplifies working with a group of `Chain*`-type objects.
To learn more, check [lXtractor documentation](https://lxtractor.readthedocs.io/en/latest/).

In [ ]:
def wrap_into_chain_seq(row):
    """
    :param row: DataFrame row.
    :return: chain sequence using "Sequence" field to obtain a full
        protein sequence and "Entry Name" for its name.
    """
    fam_df = row['Protein families']
    family = 'other'
    if 'protein kinase family' in fam_df:
        if 'Ser/Thr' in fam_df:
            family = 'STk'
        elif 'Tyr' in fam_df:
            family = 'Tk'

    return ChainSequence.from_string(
        row['Sequence'], name=row['Entry Name'], meta={'Family': family}
    )

In [ ]:
chains = ChainList(
    wrap_into_chain_seq(r) for _, r in df_up.iterrows()
)
Counter(c.meta['Family'] for c in chains)

Select only those whose family was classified as STk or Tk.

In [ ]:
chains = chains.filter(lambda x: x.meta['Family'] != 'other')

Below, alignment-extracted boundaries are used to create a "child"
subsequence for each `ChainSequence` in `chains`. We also specify acceptance
criteria: minimum length of extracted domain, minimum coverage, i.e., how many
HMM positions were covered by the extracted sequence, and minimum bit score
(which is a bit higher than the profile's own threshold; just to be safe).

In [ ]:
consume(prof.annotate(
    chains, new_map_name='PK', min_size=150, min_cov_hmm=0.7, min_score=50
));

In [ ]:
len(chains), len(chains.collapse_children())

We observe that the final number of chains with annotated domains under the
filtering criteria specified above differs from the initial number of chains.
UniProt obtains PK domain annotations using a different profile using `ProSite`
software. Thus, the filtering criteria are essentially sensitivity tweaks.

In [ ]:
chains = chains.filter(lambda x: len(x.children) > 0)
len(chains), Counter(c.meta['Family'] for c in chains)

### 2.3 Prepare encoded dataset

We encode the alignment using the ProtFP embeddings derived from the PCA of
the AAIndex database. Thus, they encapsulate many physico-chemical properties
in several compact dimensions.

The number of PCA components to take.

In [ ]:
N_COMP = 3

Define variables -- `N_COMP` PCA components for each profile position.
`lXtractor` is shipped with values already and the "calculation" of variables
below is simply accessing the table values for each valid residue. By default,
`NaN` used as a placeholder for empty values (gaps in the sequence-to-profile
alignment).

In [ ]:
variables = list(chain.from_iterable(
    (PFP(pos, i) for i in range(1, N_COMP + 1)) for pos in range(1, prof.hmm.M + 1)
))
len(variables)

In [ ]:
manager = Manager(verbose=True)
calculator = GenericCalculator()  # You may increase the number of processes here.
domains = chains.collapse_children()
df = manager.aggregate_from_it(
    # `calculate` returns a generator of the calculation results.
    # We immediately aggregate the results into a single table.
    manager.calculate(domains, variables, calculator, map_name='PK')
)

Finally, incorporate labels for STk (0) and Tk (1) domain sequences.

In [ ]:
cls_map = {'STk': 0, 'Tk': 1}
id2cls = {s.id: cls_map[s.parent.meta['Family']] for s in domains}
df['IsTk'] = df['ObjectID'].map(id2cls)

In [ ]:
df.head()

In [ ]:
df.to_csv('./fs.tmp.csv', index=False)

## 3. Run feature selection

In [ ]:
df = pd.read_csv('./fs.tmp.csv')

### 3.1 Setup

For convenience, we'll define several helper functions and a `Dataset` dataclass
holding the full dataset.

In [ ]:
from dataclasses import dataclass

import numpy as np
from eBoruta import eBoruta
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier

In [ ]:
@dataclass
class Dataset:
    df: pd.DataFrame
    x_names: list[str]
    y_name: str

    @property
    def x(self) -> pd.DataFrame:
        return self.df[self.x_names]

    @property
    def y(self) -> pd.Series:
        return self.df[self.y_name]


def sel_by_idx(ds, idx):
    """
    :param ds: Dataset.
    :param idx: Array of numeric indices.
    :return: New `Dataset` instance with the same variables
        and rows selected by `idx`.
    """
    return Dataset(ds.df.iloc[idx], ds.x_names, ds.y_name)


def score(ds, model, score_fn=f1_score):
    """
    F1-score for the classifier model.
    """
    return score_fn(ds.y.values, model.predict(ds.x))


def cv(ds, model, n=10, score_fn=f1_score, agg_fn=np.mean):
    """
    Cross validate the model returning the aggregated score.
    """
    scores = []
    splitter = StratifiedShuffleSplit(n_splits=n)
    for train_idx, test_idx in splitter.split(ds.x, ds.y):
        ds_train = sel_by_idx(ds, train_idx)
        ds_test = sel_by_idx(ds, test_idx)
        _model = model.__class__(**model.get_params())
        _model.fit(ds_train.x, ds_train.y)
        scores.append(score(ds_test, _model, score_fn))
    return agg_fn(scores)

# def plot_imp_history(df_history: pd.DataFrame):
#     sns.lineplot(x='Step', y='Importance', hue='Feature', data=df_history)
#     sns.lineplot(x='Step', y='Threshold', data=df_history, linestyle='--', linewidth=4)

In [ ]:
# How many top features to review?
TOP_N = 10

In [ ]:
dataset = Dataset(df, [c for c in df.columns if 'PFP' in c], 'IsTk')
classifier = XGBClassifier(n_jobs=-1)

### 3.2 Cross-validate the initial model

Before attempting anything, let's just first cross-validating the model on the
full dataset. We should obtain a decent result by itself. It's good practice
to test whether subsequent feature selection deteriorates this baseline result.

In [ ]:
cv(dataset, classifier)

### 3.3 Select Features

Run the feature selection itself. We'll keep the default parameters and a large
number of iterations to obtain final decisions for each feature. Then we'll rank
features and select top N features to analyze further.

In [ ]:
boruta = eBoruta(n_iter=100, shap_approximate=False, shap_tree=True)
boruta.fit(dataset.x, dataset.y, model=classifier, verbose=0)

Next, we'll rank the accepted features. Ranking means refitting the model using
supplied (here, accepted) features, calculating and sorting by feature importance
values.

In [ ]:
ranks = boruta.rank(features=boruta.features_.accepted, fit=True)
ranks.head(TOP_N)

In [ ]:
top_n_features = ranks.Feature[:TOP_N].tolist()

In [ ]:
ds_sel = Dataset(
    df[['ObjectID', *top_n_features, dataset.y_name]], top_n_features, dataset.y_name
)

### 3.4 CV score using selected features

Finally, we'll use the dataset with accepted features to cross-validate the model.
Indeed, the CV score is very similar to the previously obtained on a full dataset.

In [ ]:
cv(ds_sel, classifier)

## 4. Interpret the results

In [ ]:
POS_PATTERN = re.compile('p=(\d+)')

In [ ]:
ds_sample = ds_sel.df.groupby('IsTk').sample(30)
positions = sorted(
    {int(POS_PATTERN.findall(c)[0]) for c in ds_sample.columns if 'p=' in c}
)
positions

In [ ]:
def extract_positions(c: ChainSequence) -> str:
    m = c.get_map('PK')
    mapped = map(
        lambda x: x if x == '-' else x.seq1,
        (m.get(p, '-') for p in positions))
    return "".join(mapped)

In [ ]:
sample_chains = [domains[x].pop() for x in ds_sample['ObjectID']]

In [ ]:
positions

In [ ]:
seqs = map(extract_positions, sample_chains)

for is_tk, seq in zip(ds_sample.IsTk, seqs):
    print(seq, is_tk)

One may notice from above the following correspondence between Figures 2 and 3
and conservation patterns observed above. Firstly, position 125 clearly
corresponds to position 168 on Fig. 3, with conserved K in STk and A/R in Tk.
Furthermore, position 127 corresponds to 170 on Fig. 3, with conserved R in Tk.
Moving on, position 162 corresponds to 203 in Fig. 2, with conserved K and R in
Tk.

Thus, feature selection managed to retrieve features whose conservation patterns
differed between STk and Tk. Moreover, selected feature pertain to positions
implicated as critical for the STk/Tk functional differences by a research
paper; thus, clearly relevant.